In [4]:
import pandas as pd
df_train = pd.read_csv('../data/data_train.csv')
df_valid = pd.read_csv('../data/data_valid.csv')
df_test = pd.read_csv('../data/data_test.csv')
dataframes = [df_test, df_valid, df_train]

In [33]:
df_train = df_train.loc[:,~df_train.columns.str.startswith('Unnamed')]
df_valid = df_valid.loc[:,~df_valid.columns.str.startswith('Unnamed')]
df_test = df_test.loc[:,~df_test.columns.str.startswith('Unnamed')]

df_train = df_train.dropna(subset=['ANA', 'EXA_interpreted'], how='all')   
df_valid = df_valid.dropna(subset=['ANA', 'EXA_interpreted'], how='all')   
df_test = df_test.dropna(subset=['ANA', 'EXA_interpreted'], how='all')   


In [34]:
len(df_train) + len(df_valid) + len(df_test)

38646

## Original Data

In [35]:
def get_string(string):
    if pd.isna(string):
        return ""
    return str(string)

df_train["original_data"] = df_train.apply(
    lambda row: " ".join([
        get_string(row['age']),
        get_string(row['sex']),
        get_string(row['ANA']),
        get_string(row['EXA'])
    ]),
    axis=1
)

df_valid["original_data"] = df_valid.apply(
    lambda row: " ".join([
        get_string(row['age']),
        get_string(row['sex']),
        get_string(row['ANA']),
        get_string(row['EXA'])
    ]),
    axis=1
)

df_test["original_data"] = df_test.apply(
    lambda row: " ".join([
        get_string(row['age']),
        get_string(row['sex']),
        get_string(row['ANA']),
        get_string(row['EXA'])
    ]),
    axis=1
)

## Keywords

In [36]:
df_train["keywords_prep"] = df_train["data"].str.replace("kein ", "kein_")
df_train["keywords_prep"] = df_train["keywords_prep"].str.replace("keine ", "keine_")
df_train["keywords_prep"] = df_train["keywords_prep"].str.replace("keinen ", "keinen_")
df_train["keywords_prep"] = df_train["keywords_prep"].str.replace("nicht ", "nicht_")

df_test["keywords_prep"] = df_test["data"].str.replace("kein ", "kein_")
df_test["keywords_prep"] = df_test["keywords_prep"].str.replace("keine ", "keine_")
df_test["keywords_prep"] = df_test["keywords_prep"].str.replace("keinen ", "keinen_")
df_test["keywords_prep"] = df_test["keywords_prep"].str.replace("nicht ", "nicht_")

df_valid["keywords_prep"] = df_valid["data"].str.replace("kein ", "kein_")
df_valid["keywords_prep"] = df_valid["keywords_prep"].str.replace("keine ", "keine_")
df_valid["keywords_prep"] = df_valid["keywords_prep"].str.replace("keinen ", "keinen_")
df_valid["keywords_prep"] = df_valid["keywords_prep"].str.replace("nicht ", "nicht_")

In [37]:
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Initialize RAKE
r = Rake(include_repeated_phrases=False, min_length=2)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juliankraus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/juliankraus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [38]:
def extract_keywords(text):
    if isinstance(text, str) and text.strip() != "":
        r.extract_keywords_from_text(text)
        return " ".join(r.get_ranked_phrases())
    else:
        return ""


df_train["keywords"] = df_train["keywords_prep"].apply(extract_keywords)
df_valid["keywords"] = df_valid["keywords_prep"].apply(extract_keywords)
df_test["keywords"] = df_test["keywords_prep"].apply(extract_keywords)

In [40]:
df_test

,patientid,sex,age,ANA,EXA,DIA_text,DIA_code,OP_text,OP_code,EXA_formatted,EXA_interpreted,label,data,data2,original_data,keywords_prep,keywords
0,79097.0,male,31.0,Vor 2 1/2 Jahren vordere Kreuzbandruptur beim ...,BA: GB: re. Knie: diskret bis mäßiger Ergus...,Chronische Instabilität des Kniegelenkes: vord...,M23.51 G,NaN,NaN,ba gb rechts knie diskret bis mäßiger schwe...,Mein rechtes Knie hat eine diskrete bis mäßige...,other-minor,31.0 Jahre alt männlich Vor 2 1/2 Jahren vorde...,Vor 2 1/2 Jahren vordere Kreuzbandruptur beim ...,31.0 male Vor 2 1/2 Jahren vordere Kreuzbandru...,31.0 Jahre alt männlich Vor 2 1/2 Jahren vorde...,patient möchte joggen und wintersport machen k...
1,29121.0,female,55.0,seit ca. 1 J. schmerzhafte zystische Schwellun...,Ganglion li. lat Knie; DS lat. GS; Steinmann n...,Binnenschädigung des Kniegelenkes [internal de...,M23.- V,NaN,NaN,"druckschmerz äußerechts seite des kniegelenks,...","Mein Knie schmerzt auf der äußerechts Seite, a...",other-minor,55.0 Jahre alt weiblich seit ca. 1 J. schmerzh...,seit ca. 1 J. schmerzhafte zystische Schwellun...,55.0 female seit ca. 1 J. schmerzhafte zystisc...,55.0 Jahre alt weiblich seit ca. 1 J. schmerzh...,mein knie schmerzt auf der äußerechts seite ab...
2,76759.0,NaN,NaN,NaN,BA: gerade GB: flüssig ...,V.a.Chondromalazie medial links,M22.4 V,NaN,NaN,ba gerade gb flüssig ...,Mein linkes Knie hat keine Rötung oder Überwär...,other-minor,. Mein linkes Knie hat keine Rötung oder Überw...,. Mein linkes Knie hat keine Rötung oder Überw...,BA: gerade GB: flüssig ...,. Mein linkes Knie hat keine_Rötung oder Überw...,es gibt einen mäßigen druckschmerz auf der äuß...
3,37168.0,male,56.0,Status idem; Schmerzsymptomatik in letzter Zei...,ia-Erguß; deutliches femoro-pat. Reiben; Hyper...,Meniskusschädigung durch alten Riss oder alte ...,M23.23 V,NaN,NaN,"ia-erguß, druckschmerz med gel",Ich habe Schmerzen auf der medialen Seite des ...,meniskus-minor,56.0 Jahre alt männlich Status idem; Schmerzsy...,Status idem; Schmerzsymptomatik in letzter Zei...,56.0 male Status idem; Schmerzsymptomatik in l...,56.0 Jahre alt männlich Status idem; Schmerzsy...,ich habe schmerzen auf der medialen seite des ...
4,67753.0,female,31.0,Heute Vorstellung mit MRT Bildern vom li Knie.,li. Knie: 0/0/135; klinisch kein Gelenkerguss;...,Z.n.Patellaluxation links,S83.0 Z,Arthroskopische Refixation und Plastik am Kaps...,5-813.9,links knie keine überstreckung des knies kein ...,Mein linkes Knie kann nicht überstreckt werden...,other-minor,31.0 Jahre alt weiblich Heute Vorstellung mit ...,Heute Vorstellung mit MRT Bildern vom li Knie....,31.0 female Heute Vorstellung mit MRT Bildern ...,31.0 Jahre alt weiblich Heute Vorstellung mit ...,0 jahre alt weiblich heute vorstellung mit mrt...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7837,13709.0,male,63.0,Zwischenanamnese: Der Patient gibt an St.n. OS...,Klinischer Befund des Sprunggelenks re. OSG en...,Chondromalacia patellae,M22.4 G,NaN,NaN,druckschmerz über ventroäußeree gelenkkapsel,Ich habe Schmerzen an der ventroäußeren Seite ...,other-minor,63.0 Jahre alt männlich Zwischenanamnese: Der ...,Zwischenanamnese: Der Patient gibt an St.n. OS...,63.0 male Zwischenanamnese: Der Patient gibt a...,63.0 Jahre alt männlich Zwischenanamnese: Der ...,0 jahre alt männlich zwischenanamnese der vent...
7838,66956.0,male,40.0,kommt mit aktuellen MRT Bildern,NaN,V.a.retropatellare Chondromalazie rechts,M22.4 V,NaN,NaN,NaN,NaN,other-minor,40.0 Jahre alt männlich kommt mit aktuellen MR...,kommt mit aktuellen MRT Bildern. nan,40.0 male kommt mit aktuellen MRT Bildern,40.0 Jahre alt männlich kommt mit aktuellen MR...,0 jahre alt männlich kommt mit aktuellen mrt b...
7839,63044.0,female,60.0,NaN,re Schulter: Wunden reizl. keine Rötung; keine...,Binnenschädigung des Kniegelenkes; nicht näher...,M23.90 Z,Arthroskopische Gelenkoperation: Entfernung pe...,5-810.50,keine rötung,Mein Knie ist nicht gerötet.,meniskus-minor,60.0 Jahre alt weiblich . Me

In [39]:
df_train.to_csv('../data/data_train.csv', index=False)
df_valid.to_csv('../data/data_valid.csv', index=False)
df_test.to_csv('../data/data_test.csv', index=False)